In [10]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import requests
import pandas as pd
import numpy as np

In [52]:
driver=webdriver.Chrome()
driver.get("https://www.goodchoice.kr/product/detail?ano=68834")
time.sleep(5)
driver.find_element_by_class_name("tab_review").click()
time.sleep(5)

In [54]:
review_crawling=[]

while True:
    try:
        page_bar = driver.find_element_by_class_name('paging')
        pages = page_bar.find_elements_by_tag_name('button')
        try:
            if isinstance(int(pages[0].text.strip()),int) == True:
                page_now = pages[0].text.strip()
        except: page_now = pages[1].text.strip()
            
        for page in pages:
            if page.text == '이전':
                pass
            elif page.text == '다음':
                page.send_keys("\n")
                time.sleep(5)
            elif int(page.text.strip()) > int(page_now):
                page.send_keys("\n")
                time.sleep(5)
                
            html = driver.page_source
            soup = bs(html,'lxml')
            reviews=soup.select("#review > ul > li")
            for review in reviews:
                try:
                    user_name=review.select_one("div.name").get_text().split(" · ")[1].split("\n")[0]
                except:
                    user_name=None
                point=review.select_one("div.num").get_text()
                date=review.select_one("span.date").get_text()
                txt=review.select_one("div.txt").get_text()
                review_crawling.append([user_name,point,date,txt])
    except:
        break

In [55]:
len(review_crawling)

145

In [56]:
review_df=pd.DataFrame(review_crawling,columns=['user_name','point','date','txt'])
review_df=review_df.drop_duplicates(['txt'],keep = 'first')
review_df.shape

(94, 4)

In [57]:
review_df.to_csv("C:\\Users\\hun98\\crawling\\result\\goodchoice_henna.csv")